In [1]:
# source: https://fgc.stpi.narl.org.tw/activity/videoDetail/4b1141305ddf5522015de5479f4701b1
# pip install jieba
import numpy as np

# 建立 詞向量 dict
dim = 0
word_vecs= {}
# 開啟詞向量檔案 注意要去改你作業系統中該檔案的位置
with open('C:/python/20210906-Python-第三階段-13/materials/eppy1109it2/code/cna.cbow.cwe_p.tar_g.512d.0.txt', encoding='utf-8') as f:
  for line in f:
    # 假設我們的詞向量有300維
    # 由word以及向量中的元素共301個
    # 以空格分隔組成詞向量檔案中一行
    tokens = line.strip().split()

    # 第一行是兩個整數，分別代表有幾個詞向量，以及詞向量維度
    if len(tokens) == 2:
      dim = int(tokens[1])
      continue
    
    word = tokens[0] 
    vec = np.array([ float(t) for t in tokens[1:] ])
    word_vecs[word] = vec
    

In [6]:
%%time
import jieba
import numpy as np

# 我們要從answers中挑出應該接在dialogue之後的短句
dialogue = "如果飛機在飛行當中打一個小洞的話 會不會影響飛行的安全呢"
answers = [
  "其實狗搖尾巴有很多種方式 高興搖尾巴 生氣也搖尾巴",  
  "如果這個洞的話經過仔細的設計的話 應該不至於造成太大問題",
  "所以只要依照政府規定 在採收前十天不要噴灑農藥", 
  "靜電才是加油站爆炸的元凶 手機不過是代罪羔羊",
  "我們可以用表面張力及附著力的原理 來測試看看",
  "不過蝦子死亡後 身體會釋放出有毒素的體液 可能造成水的變質"]

emb_cnt = 0
avg_dlg_emb = np.zeros((dim,))
# jieba.cut 會把dialogue作分詞
# 對於有在word_vecs裡面的詞我們才把它取出
# 最後詞向量加總取平均，作為句子的向量表示
for word in jieba.cut(dialogue):
  if word in word_vecs:
    avg_dlg_emb += word_vecs[word]
    emb_cnt += 1
avg_dlg_emb /= emb_cnt

emb_cnt = 0
max_idx = -1
max_sim = -10
# 在六個回答中，每個答句都取詞向量平均作為向量表示
# 我們選出與dialogue句子向量表示cosine similarity最高的短句
for idx,ans in enumerate(answers):
  # 六個回答維度均設為 dim (訓練資料的維度)
  avg_ans_emb = np.zeros((dim,))
  for word in jieba.cut(ans):
    if word in word_vecs:
      avg_ans_emb += word_vecs[word]
      emb_cnt += 1
  # 計算 cosine 相似性    
  sim = np.dot(avg_dlg_emb, avg_ans_emb) / np.linalg.norm(avg_dlg_emb) / np.linalg.norm(avg_ans_emb)
  print("#%d 相似性: %f" % (idx, sim))
  if sim > max_sim:
    max_idx = idx
    max_sim = sim

print('問: '+ dialogue)    
print("答: " + answers[max_idx])    
print("答: %d" % max_idx)    

#0 相似性: 0.403162
#1 相似性: 0.295816
#2 相似性: 0.216000
#3 相似性: 0.247240
#4 相似性: 0.423816
#5 相似性: 0.846152
問: 身體會釋放出有毒素的體液
答: 不過蝦子死亡後 身體會釋放出有毒素的體液 可能造成水的變質
答: 5
Wall time: 4.99 ms
